In [1]:
import numpy as np, sys,os
import pandas as pd
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm
sys.path.insert(0, "..")

In [2]:
df = pd.read_json('../data/processed/realData.json', orient='records', lines=True)


In [3]:
df = pd.read_json('../data/processed/realData.json', orient='records', lines=True)
# Convertir las cadenas en listas de números
df['cop_x'] = df['cop_x'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))
df['cop_y'] = df['cop_y'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))

# Convertir las series de tiempo a arrays numpy
X_cop_x = np.array(df['cop_x'].tolist())
X_cop_y = np.array(df['cop_y'].tolist())
X = np.stack((X_cop_x, X_cop_y), axis=-1)  # Combinar en una sola entrada con 2 canales

# Convertir las etiquetas a números
label_map = {'Healthy': 0, 'Diabetic': 1, 'Neuropathic': 2}
y = df['class'].map(label_map).values

# Crear etiquetas para nivel 1 y nivel 2
level1_labels = (y > 0).astype(int)  # 0: Healthy, 1: Diabetic (incluye Neuropathic)
level2_labels = (y == 2).astype(int)  # 0: Diabetic sin neuropatía, 1: Neuropathic


In [4]:
from sklearn.model_selection import train_test_split

# Separar los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear etiquetas para nivel 1 y nivel 2 para el conjunto de prueba
level1_labels_train = (y_train > 0).astype(int)
level2_labels_train = (y_train == 2).astype(int)

level1_labels_test = (y_test > 0).astype(int)
level2_labels_test = (y_test == 2).astype(int)


In [7]:
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size=3, stride=1):
    conv1 = Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    bn1 = BatchNormalization()(conv1)
    act1 = Activation('relu')(bn1)

    conv2 = Conv1D(filters, kernel_size, strides=1, padding='same')(act1)
    bn2 = BatchNormalization()(conv2)

    if stride != 1:
        x = Conv1D(filters, 1, strides=stride, padding='same')(x)
    shortcut = Add()([x, bn2])
    output = Activation('relu')(shortcut)
    return output

input_shape = X.shape[1:]
inputs = Input(shape=input_shape)

x = Conv1D(64, 7, strides=2, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = residual_block(x, 64)
x = residual_block(x, 64)
x = residual_block(x, 64)

x = GlobalAveragePooling1D()(x)

# Salida para nivel 1 (Healthy vs Diabetic)
level1_output = Dense(1, activation='sigmoid', name='level1_output')(x)

# Salida para nivel 2 (Diabetic sin neuropatía vs Neuropathic)
level2_output = Dense(1, activation='sigmoid', name='level2_output')(x)

model = Model(inputs=inputs, outputs=[level1_output, level2_output])

model.compile(optimizer='adam', 
              loss={'level1_output': 'binary_crossentropy', 'level2_output': 'binary_crossentropy'},
              metrics={'level1_output': 'accuracy', 'level2_output': 'accuracy'})
model.summary()

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 500, 2)    │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_7 (Conv1D)   │ (None, 250, 64)   │        960 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 64)   │        256 │ conv1d_7[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_7        │ (None, 250, 64)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_8 (Conv1D)   │ (None, 250, 64)   │     12,352 │ activation_7[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 64)   │        256 │ conv1d_8[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_8        │ (None, 250, 64)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_9 (Conv1D)   │ (None, 250, 64)   │     12,352 │ activation_8[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 64)   │        256 │ conv1d_9[0][0]    │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 250, 64)   │          0 │ activation_7[0][… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_9        │ (None, 250, 64)   │          0 │ add_3[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_10 (Conv1D)  │ (None, 250, 64)   │     12,352 │ activation_9[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 64)   │        256 │ conv1d_10[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_10       │ (None, 250, 64)   │          0 │ batch_normalizat… │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv1d_11 (Conv1D)  │ (None, 250, 64)   │     12,352 │ activation_10[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 250, 64)   │        256 │ conv1d_11[0][0]   │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_4 (Add)         │ (None, 250, 64)   │          0 │ activation_9[0][… │
│                     │                   │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ activation_11       │ (None, 250, 64)   │          0 │ add_4[0][0]       │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 76,994 (300.76 KB)

 Trainable params: 76,098 (297.26 KB)

 Non-trainable params: 896 (3.50 KB)

In [6]:
model.fit(X_train, {'level1_output': level1_labels_train, 'level2_output': level2_labels_train}, 
          epochs=100, batch_size=32, validation_split=0.2)


Epoch 1/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 147ms/step - level1_output_accuracy: 0.4422 - level2_output_accuracy: 0.5500 - loss: 1.6860 - val_level1_output_accuracy: 0.8000 - val_level2_output_accuracy: 0.6714 - val_loss: 1.3209
Epoch 2/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step - level1_output_accuracy: 0.8468 - level2_output_accuracy: 0.6334 - loss: 1.0451 - val_level1_output_accuracy: 0.8000 - val_level2_output_accuracy: 0.6000 - val_loss: 1.2766
Epoch 3/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 71ms/step - level1_output_accuracy: 0.8338 - level2_output_accuracy: 0.7032 - loss: 1.0448 - val_level1_output_accuracy: 0.8000 - val_level2_output_accuracy: 0.5857 - val_loss: 1.2708
Epoch 4/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 76ms/step - level1_output_accuracy: 0.8124 - level2_output_accuracy: 0.6674 - loss: 1.0328 - val_level1_output_accuracy: 0.7857 - val_level2_output_accuracy: 0.5857 - val_loss: 1.2887
Epoch 5/100
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 69ms/step - level1_output_accuracy: 0.8447 - level2_output_accu

In [17]:
# Hacer predicciones con el conjunto de prueba
predictions = model.predict(X_test)

# Extraer las predicciones de nivel 1 y nivel 2
level1_predictions = (predictions[0] > 0.5).astype(int)
level2_predictions = (predictions[1] > 0.5).astype(int)

# Evaluar el rendimiento del modelo
from sklearn.metrics import accuracy_score, classification_report

# Evaluación de nivel 1
level1_accuracy = accuracy_score(level1_labels_test, level1_predictions)
print(f'Accuracy for Level 1 (Healthy vs Diabetic): {level1_accuracy}')
print(classification_report(level1_labels_test, level1_predictions, target_names=['Healthy', 'Diabetic']))

# Evaluación de nivel 2 (solo para las muestras clasificadas como Diabetic)
diabetic_indices = np.where(level1_labels_test == 1)[0]
level2_accuracy = accuracy_score(level2_labels_test[diabetic_indices], level2_predictions[diabetic_indices])
print(f'Accuracy for Level 2 (Diabetic without neuropathy vs Neuropathic): {level2_accuracy}')
print(classification_report(level2_labels_test[diabetic_indices], level2_predictions[diabetic_indices], target_names=['Diabetic without neuropathy', 'Neuropathic']))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step 
Accuracy for Level 1 (Healthy vs Diabetic): 0.7701149425287356
              precision    recall  f1-score   support

     Healthy       0.65      0.50      0.57        26
    Diabetic       0.81      0.89      0.84        61

    accuracy                           0.77        87
   macro avg       0.73      0.69      0.70        87
weighted avg       0.76      0.77      0.76        87

Accuracy for Level 2 (Diabetic without neuropathy vs Neuropathic): 0.5081967213114754
                             precision    recall  f1-score   support

Diabetic without neuropathy       0.43      0.96      0.59        23
                Neuropathic       0.90      0.24      0.38        38

                   accuracy                           0.51        61
                  macro avg       0.67      0.60      0.48        61
               weighted avg       0.72      0.51      0.46        61



In [22]:

df = pd.read_json('../data/processed/realData.json', orient='records', lines=True)
# Convertir las cadenas en listas de números
df['cop_x'] = df['cop_x'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))
df['cop_y'] = df['cop_y'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))

# Convertir las series de tiempo a arrays numpy
X_cop_x = np.array(df['cop_x'].tolist())
X_cop_y = np.array(df['cop_y'].tolist())
X = np.stack((X_cop_x, X_cop_y), axis=-1)  # Combinar en una sola entrada con 2 canales

# Convertir las etiquetas a números
label_map = {'Healthy': 0, 'Diabetic': 1, 'Neuropathic': 2}
y = df['class'].map(label_map).values

# Crear etiquetas para nivel 1 y nivel 2
level1_labels = (y > 0).astype(int)  # 0: Healthy, 1: Diabetic (incluye Neuropathic)
level2_labels = (y == 2).astype(int)  # 0: Diabetic sin neuropatía, 1: Neuropathic


In [23]:
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size=3, stride=1):
    conv1 = Conv1D(filters, kernel_size, strides=stride, padding='same')(x)
    bn1 = BatchNormalization()(conv1)
    act1 = Activation('relu')(bn1)

    conv2 = Conv1D(filters, kernel_size, strides=1, padding='same')(act1)
    bn2 = BatchNormalization()(conv2)

    if stride != 1:
        x = Conv1D(filters, 1, strides=stride, padding='same')(x)
    shortcut = Add()([x, bn2])
    output = Activation('relu')(shortcut)
    return output

input_shape = X.shape[1:]
inputs = Input(shape=input_shape)

x = Conv1D(64, 7, strides=2, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = residual_block(x, 64)
x = residual_block(x, 64)
x = residual_block(x, 64)

x = GlobalAveragePooling1D()(x)

# Salida para nivel 1 (Healthy vs Diabetic)
level1_output = Dense(1, activation='sigmoid', name='level1_output')(x)

# Salida para nivel 2 (Diabetic sin neuropatía vs Neuropathic)
level2_output = Dense(1, activation='sigmoid', name='level2_output')(x)

model = Model(inputs=inputs, outputs=[level1_output, level2_output])

model.compile(optimizer='adam', 
              loss={'level1_output': 'binary_crossentropy', 'level2_output': 'binary_crossentropy'},
              metrics={'level1_output': 'accuracy', 'level2_output': 'accuracy'})


In [20]:
model.fit(X, {'level1_output': level1_labels, 'level2_output': level2_labels}, 
          epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 6s 93ms/step - level1_output_accuracy: 0.5668 - level2_output_accuracy: 0.6873 - loss: 1.2816 - val_level1_output_accuracy: 0.9425 - val_level2_output_accuracy: 0.2874 - val_loss: 1.3355
Epoch 2/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 56ms/step - level1_output_accuracy: 0.7329 - level2_output_accuracy: 0.7481 - loss: 1.0167 - val_level1_output_accuracy: 0.9770 - val_level2_output_accuracy: 0.0000e+00 - val_loss: 1.3933
Epoch 3/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 58ms/step - level1_output_accuracy: 0.7924 - level2_output_accuracy: 0.7312 - loss: 1.0011 - val_level1_output_accuracy: 0.8736 - val_level2_output_accuracy: 0.0000e+00 - val_loss: 1.4532
Epoch 4/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 78ms/step - level1_output_accuracy: 0.7668 - level2_output_accuracy: 0.7165 - loss: 1.0643 - val_level1_output_accuracy: 0.9195 - val_level2_output_accuracy: 0.0000e+00 - val_loss: 1.4695
Epoch 5/10
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 57ms/step - level1_output_accuracy: 0.7787 - le

In [24]:
from sklearn.model_selection import train_test_split

# Separar los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Crear etiquetas para nivel 1 y nivel 2 para el conjunto de prueba
level1_labels_train = (y_train > 0).astype(int)
level2_labels_train = (y_train == 2).astype(int)

level1_labels_test = (y_test > 0).astype(int)
level2_labels_test = (y_test == 2).astype(int)

# Entrenar el modelo (si no se ha hecho anteriormente)
model.fit(X_train, {'level1_output': level1_labels_train, 'level2_output': level2_labels_train}, 
          epochs=10, batch_size=32, validation_split=0.2)

# Hacer predicciones con el conjunto de prueba
predictions = model.predict(X_test)

# Extraer las predicciones de nivel 1 y nivel 2
level1_predictions = (predictions[0] > 0.5).astype(int)
level2_predictions = (predictions[1] > 0.5).astype(int)

# Evaluar el rendimiento del modelo
from sklearn.metrics import accuracy_score, classification_report

# Evaluación de nivel 1
level1_accuracy = accuracy_score(level1_labels_test, level1_predictions)
print(f'Accuracy for Level 1 (Healthy vs Diabetic): {level1_accuracy}')
print(classification_report(level1_labels_test, level1_predictions, target_names=['Healthy', 'Diabetic']))

# Evaluación de nivel 2 (solo para las muestras clasificadas como Diabetic)
diabetic_indices = np.where(level1_labels_test == 1)[0]
level2_accuracy = accuracy_score(level2_labels_test[diabetic_indices], level2_predictions[diabetic_indices])
print(f'Accuracy for Level 2 (Diabetic without neuropathy vs Neuropathic): {level2_accuracy}')
print(classification_report(level2_labels_test[diabetic_indices], level2_predictions[diabetic_indices], target_names=['Diabetic without neuropathy', 'Neuropathic']))


Epoch 1/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 5s 92ms/step - level1_output_accuracy: 0.7378 - level2_output_accuracy: 0.5965 - loss: 1.2156 - val_level1_output_accuracy: 0.8000 - val_level2_output_accuracy: 0.5714 - val_loss: 1.3130
Epoch 2/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - level1_output_accuracy: 0.8189 - level2_output_accuracy: 0.6451 - loss: 1.0820 - val_level1_output_accuracy: 0.8000 - val_level2_output_accuracy: 0.5571 - val_loss: 1.2847
Epoch 3/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - level1_output_accuracy: 0.8291 - level2_output_accuracy: 0.7066 - loss: 0.9866 - val_level1_output_accuracy: 0.8000 - val_level2_output_accuracy: 0.5714 - val_loss: 1.2900
Epoch 4/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 55ms/step - level1_output_accuracy: 0.8403 - level2_output_accuracy: 0.7421 - loss: 0.9397 - val_level1_output_accuracy: 0.7714 - val_level2_output_accuracy: 0.5714 - val_loss: 1.2615
Epoch 5/10
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - level1_output_accuracy: 0.8499 - level2_output_accuracy: 

In [25]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.regularizers import l2

# Cargar los datos
df = pd.read_json('../data/processed/realData.json', orient='records', lines=True)

# Convertir las cadenas en listas de números
df['cop_x'] = df['cop_x'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))
df['cop_y'] = df['cop_y'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))

# Normalización
scaler_x = StandardScaler()
scaler_y = StandardScaler()

# Ajustar y transformar las series de tiempo
X_cop_x = np.array(df['cop_x'].tolist())
X_cop_y = np.array(df['cop_y'].tolist())

X_cop_x = scaler_x.fit_transform(X_cop_x)
X_cop_y = scaler_y.fit_transform(X_cop_y)

X = np.stack((X_cop_x, X_cop_y), axis=-1)

# Convertir las etiquetas a números
label_map = {'Healthy': 0, 'Diabetic': 1, 'Neuropathic': 2}
y = df['class'].map(label_map).values

# Crear etiquetas para nivel 1 y nivel 2
level1_labels = (y > 0).astype(int)  # 0: Healthy, 1: Diabetic (incluye Neuropathic)
level2_labels = (y == 2).astype(int)  # 0: Diabetic sin neuropatía, 1: Neuropathic

# Separar los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
level1_labels_train = (y_train > 0).astype(int)
level2_labels_train = (y_train == 2).astype(int)
level1_labels_test = (y_test > 0).astype(int)
level2_labels_test = (y_test == 2).astype(int)

# Construcción del modelo ResNet mejorado
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model

def residual_block(x, filters, kernel_size=3, stride=1, dropout_rate=0.3):
    conv1 = Conv1D(filters, kernel_size, strides=stride, padding='same', kernel_regularizer=l2(1e-4))(x)
    bn1 = BatchNormalization()(conv1)
    act1 = Activation('relu')(bn1)
    drop1 = Dropout(dropout_rate)(act1)

    conv2 = Conv1D(filters, kernel_size, strides=1, padding='same', kernel_regularizer=l2(1e-4))(drop1)
    bn2 = BatchNormalization()(conv2)

    if stride != 1:
        x = Conv1D(filters, 1, strides=stride, padding='same')(x)
    shortcut = Add()([x, bn2])
    output = Activation('relu')(shortcut)
    return output

input_shape = X.shape[1:]
inputs = Input(shape=input_shape)

x = Conv1D(64, 7, strides=2, padding='same')(inputs)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = residual_block(x, 64)
x = residual_block(x, 64)
x = residual_block(x, 64)

x = GlobalAveragePooling1D()(x)

# Salida para nivel 1 (Healthy vs Diabetic)
level1_output = Dense(1, activation='sigmoid', name='level1_output')(x)

# Salida para nivel 2 (Diabetic sin neuropatía vs Neuropathic)
level2_output = Dense(1, activation='sigmoid', name='level2_output')(x)

model = Model(inputs=inputs, outputs=[level1_output, level2_output])

model.compile(optimizer='adam', 
              loss={'level1_output': 'binary_crossentropy', 'level2_output': 'binary_crossentropy'},
              metrics={'level1_output': 'accuracy', 'level2_output': 'accuracy'})

# Implementar Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Entrenar el modelo
model.fit(X_train, {'level1_output': level1_labels_train, 'level2_output': level2_labels_train}, 
          epochs=50, batch_size=32, validation_split=0.1, callbacks=[early_stopping])


Epoch 1/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 145ms/step - level1_output_accuracy: 0.8414 - level2_output_accuracy: 0.5986 - loss: 1.1788 - val_level1_output_accuracy: 0.8000 - val_level2_output_accuracy: 0.5714 - val_loss: 1.3826
Epoch 2/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - level1_output_accuracy: 0.8403 - level2_output_accuracy: 0.6470 - loss: 1.0671 - val_level1_output_accuracy: 0.7000 - val_level2_output_accuracy: 0.5857 - val_loss: 1.3819
Epoch 3/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - level1_output_accuracy: 0.8408 - level2_output_accuracy: 0.7244 - loss: 1.0111 - val_level1_output_accuracy: 0.6857 - val_level2_output_accuracy: 0.5571 - val_loss: 1.3714
Epoch 4/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 66ms/step - level1_output_accuracy: 0.8730 - level2_output_accuracy: 0.6571 - loss: 0.9978 - val_level1_output_accuracy: 0.6429 - val_level2_output_accuracy: 0.5143 - val_loss: 1.3895
Epoch 5/50
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 68ms/step - level1_output_accuracy: 0.8032 - level2_output_accuracy:

In [26]:
# Hacer predicciones con el conjunto de prueba
predictions = model.predict(X_test)

# Extraer las predicciones de nivel 1 y nivel 2
level1_predictions = (predictions[0] > 0.5).astype(int)
level2_predictions = (predictions[1] > 0.5).astype(int)

# Evaluar el rendimiento del modelo
from sklearn.metrics import accuracy_score, classification_report

# Evaluación de nivel 1
level1_accuracy = accuracy_score(level1_labels_test, level1_predictions)
print(f'Accuracy for Level 1 (Healthy vs Diabetic): {level1_accuracy}')
print(classification_report(level1_labels_test, level1_predictions, target_names=['Healthy', 'Diabetic']))

# Evaluación de nivel 2 (solo para las muestras clasificadas como Diabetic)
diabetic_indices = np.where(level1_labels_test == 1)[0]
level2_accuracy = accuracy_score(level2_labels_test[diabetic_indices], level2_predictions[diabetic_indices])
print(f'Accuracy for Level 2 (Diabetic without neuropathy vs Neuropathic): {level2_accuracy}')
print(classification_report(level2_labels_test[diabetic_indices], level2_predictions[diabetic_indices], target_names=['Diabetic without neuropathy', 'Neuropathic']))


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Accuracy for Level 1 (Healthy vs Diabetic): 0.6781609195402298
              precision    recall  f1-score   support

     Healthy       0.25      0.04      0.07        26
    Diabetic       0.70      0.95      0.81        61

    accuracy                           0.68        87
   macro avg       0.47      0.49      0.44        87
weighted avg       0.56      0.68      0.58        87

Accuracy for Level 2 (Diabetic without neuropathy vs Neuropathic): 0.5573770491803278
                             precision    recall  f1-score   support

Diabetic without neuropathy       0.46      0.96      0.62        23
                Neuropathic       0.92      0.32      0.47        38

                   accuracy                           0.56        61
                  macro avg       0.69      0.64      0.55        61
               weighted avg       0.75      0.56      0.53        61



In [28]:
import kerastuner as kt
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model

def build_model(hp):
    input_shape = X.shape[1:]
    inputs = Input(shape=input_shape)

    x = Conv1D(64, 7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Definir los bloques residuales con hiperparámetros
    for i in range(hp.Int('num_blocks', 2, 4)):
        filters = hp.Int('filters_' + str(i), min_value=32, max_value=128, step=32)
        kernel_size = hp.Choice('kernel_size_' + str(i), values=[3, 5, 7])
        x = residual_block(x, filters, kernel_size=kernel_size)

    x = GlobalAveragePooling1D()(x)

    level1_output = Dense(1, activation='sigmoid', name='level1_output')(x)

    level2_output = Dense(1, activation='sigmoid', name='level2_output')(x)

    model = Model(inputs=inputs, outputs=[level1_output, level2_output])

    model.compile(optimizer='adam', 
                  loss={'level1_output': 'binary_crossentropy', 'level2_output': 'binary_crossentropy'},
                  metrics={'level1_output': 'accuracy', 'level2_output': 'accuracy'})
    return model

def residual_block(x, filters, kernel_size=3, stride=1):
    conv1 = Conv1D(filters, kernel_size, strides=stride, padding='same', kernel_regularizer=l2(1e-4))(x)
    bn1 = BatchNormalization()(conv1)
    act1 = Activation('relu')(bn1)
    drop1 = Dropout(0.3)(act1)

    conv2 = Conv1D(filters, kernel_size, strides=1, padding='same', kernel_regularizer=l2(1e-4))(drop1)
    bn2 = BatchNormalization()(conv2)

    if stride != 1:
        x = Conv1D(filters, 1, strides=stride, padding='same')(x)
    shortcut = Add()([x, bn2])
    output = Activation('relu')(shortcut)
    return output



C:\Users\nicol\AppData\Local\Temp\ipykernel_16012\983189618.py:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [9]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

df = pd.read_json('../data/processed/realData.json', orient='records', lines=True)


# Convertir las cadenas en listas de números (asumiendo que 'cop_x' y 'cop_y' son listas de números)
df['cop_x'] = df['cop_x'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))
df['cop_y'] = df['cop_y'].apply(lambda x: np.array(eval(x)) if isinstance(x, str) else np.array(x))

# Normalización de las series de tiempo
scaler_x = StandardScaler()
scaler_y = StandardScaler()

X_cop_x = np.array(df['cop_x'].tolist())
X_cop_y = np.array(df['cop_y'].tolist())

X_cop_x = scaler_x.fit_transform(X_cop_x)
X_cop_y = scaler_y.fit_transform(X_cop_y)

X = np.stack((X_cop_x, X_cop_y), axis=-1)

# Mapeo de las etiquetas a números (asumiendo clases: 'Healthy', 'Diabetic', 'Neuropathic')
label_map = {'Healthy': 0, 'Diabetic': 1, 'Neuropathic': 2}
y = df['class'].map(label_map).values

# Crear etiquetas para nivel 1 y nivel 2
level1_labels = (y > 0).astype(int)  # 0: Healthy, 1: Diabetic (incluye Neuropathic)
level2_labels = (y == 2).astype(int)  # 0: Diabetic sin neuropatía, 1: Neuropathic

# Separar los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
level1_labels_train = (y_train > 0).astype(int)
level2_labels_train = (y_train == 2).astype(int)
level1_labels_test = (y_test > 0).astype(int)
level2_labels_test = (y_test == 2).astype(int)


In [11]:
from kerastuner import Objective, RandomSearch
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Add, GlobalAveragePooling1D, Dense
from tensorflow.keras.models import Model
import kerastuner as kt

# Define la función para construir el modelo
def build_model(hp):
    input_shape = X_train.shape[1:]
    inputs = Input(shape=input_shape)

    x = Conv1D(64, 7, strides=2, padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Definir los bloques residuales con hiperparámetros
    for i in range(hp.Int('num_blocks', 2, 4)):
        filters = hp.Int('filters_' + str(i), min_value=32, max_value=128, step=32)
        kernel_size = hp.Choice('kernel_size_' + str(i), values=[3, 5, 7])
        x = residual_block(x, filters=filters, kernel_size=kernel_size)

    x = GlobalAveragePooling1D()(x)

    # Salida para nivel 1 (Healthy vs Diabetic)
    level1_output = Dense(1, activation='sigmoid', name='level1_output')(x)

    # Salida para nivel 2 (Diabetic sin neuropatía vs Neuropathic)
    level2_output = Dense(1, activation='sigmoid', name='level2_output')(x)

    model = Model(inputs=inputs, outputs=[level1_output, level2_output])

    model.compile(optimizer='adam', 
                  loss={'level1_output': 'binary_crossentropy', 'level2_output': 'binary_crossentropy'},
                  metrics={'level1_output': 'accuracy', 'level2_output': 'accuracy'})
    return model

def residual_block(x, filters, kernel_size=3):
    conv1 = Conv1D(filters, kernel_size, strides=1, padding='same')(x)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)

    conv2 = Conv1D(filters, kernel_size, strides=1, padding='same')(conv1)
    conv2 = BatchNormalization()(conv2)

    shortcut = Conv1D(filters, 1, strides=1, padding='same')(x)
    shortcut = BatchNormalization()(shortcut)

    x = Add()([shortcut, conv2])
    x = Activation('relu')(x)
    return x

# Configurar el optimizador de búsqueda aleatoria y la métrica
tuner = RandomSearch(
    build_model,
    objective=Objective('val_level1_output_accuracy', direction='max'),  # Especificar la métrica y dirección de optimización
    max_trials=20,  # Número de combinaciones de hiperparámetros a probar
    directory='my_dir',
    project_name='resnet_tuning'
)

# Realizar la búsqueda de hiperparámetros
tuner.search(X_train, {'level1_output': level1_labels_train, 'level2_output': level2_labels_train},
             epochs=10,  # Ajusta este valor según sea necesario
             validation_split=0.2)  # Ajusta el tamaño de la partición de validación según sea necesario


Trial 20 Complete [00h 00m 14s]
val_level1_output_accuracy: 0.8142856955528259

Best val_level1_output_accuracy So Far: 0.8142856955528259
Total elapsed time: 00h 06m 29s


In [12]:
# Obtener los mejores hiperparámetros encontrados
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Construir el modelo final con los mejores hiperparámetros
model = build_model(best_hps)


In [13]:
from tensorflow.keras.callbacks import EarlyStopping

# Entrenar el modelo final
history = model.fit(X_train, {'level1_output': level1_labels_train, 'level2_output': level2_labels_train},
                    epochs=100,  # Ajusta el número de épocas según sea necesario
                    validation_split=0.1,  # Ajusta el tamaño de la partición de validación según sea necesario
                    callbacks=[EarlyStopping(patience=15, restore_best_weights=True)])


Epoch 1/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 11s 232ms/step - level1_output_accuracy: 0.7666 - level2_output_accuracy: 0.5842 - loss: 1.1875 - val_level1_output_accuracy: 0.7429 - val_level2_output_accuracy: 0.4857 - val_loss: 1.3632
Epoch 2/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 164ms/step - level1_output_accuracy: 0.8242 - level2_output_accuracy: 0.6738 - loss: 1.0316 - val_level1_output_accuracy: 0.8571 - val_level2_output_accuracy: 0.5714 - val_loss: 1.3024
Epoch 3/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 160ms/step - level1_output_accuracy: 0.8319 - level2_output_accuracy: 0.6681 - loss: 0.9769 - val_level1_output_accuracy: 0.8571 - val_level2_output_accuracy: 0.6571 - val_loss: 1.2991
Epoch 4/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 158ms/step - level1_output_accuracy: 0.8425 - level2_output_accuracy: 0.7227 - loss: 0.9257 - val_level1_output_accuracy: 0.6857 - val_level2_output_accuracy: 0.5429 - val_loss: 1.3212
Epoch 5/100
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 158ms/step - level1_output_accuracy: 0.8457 - lev

In [14]:
eval_results = model.evaluate(X_test, {'level1_output': level1_labels_test, 'level2_output': level2_labels_test})
print("\nEvaluation results:")
for metric, value in zip(model.metrics_names, eval_results):
    print(f"{metric}: {value}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - level1_output_accuracy: 0.7529 - level2_output_accuracy: 0.6263 - loss: 1.2172

Evaluation results:
loss: 1.2040011882781982
compile_metrics: 0.7011494040489197


Trial 2 Complete [00h 00m 01s]

Best val_acc So Far: None
Total elapsed time: 00h 00m 03s

Search: Running Trial #3

Value             |Best Value So Far |Hyperparameter
2                 |2                 |num_blocks
128               |128               |filters_0
32                |32                |filters_1
64                |32                |filters_2
32                |128               |filters_3
7                 |3                 |kernel_size_0
7                 |5                 |kernel_size_1
7                 |7                 |kernel_size_2
5                 |3                 |kernel_size_3
0.0001            |0.001             |learning_rate



Traceback (most recent call last):
  File "c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
 

RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\nicol\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras_tuner\src\engine\tuner.py